In [ ]:
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes,
    Graph
)
from semantic_mpc_interface.utils import query_to_df
from semantic_mpc_interface.namespaces import BRICK, RDF, S223, A, HPFS
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 'brick'
# base path
base_path = f'{ontology}'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


In [19]:
# Please disregard excessive outputs (logging and warnings) from bmotif
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

Generating shapes "mints" our templates into an ontology 

The templates describe how certain things should be semantically represented in our graph. 
We can fill the templates to create these semantic representations.
However, using the SHACLHandler we can use the rules of the s223 or brick ontologies to invert the templates, and instead create validation rules looking for things represented like our templates.

The templates will be added as classes to a shacl graph, currently using the namespace HPFS (urn:hpflex/shapes), and we will infer what things in the graph match our templates. 

In [ ]:
handler = SHACLHandler(ontology=ontology, template_dir = '/Users/lazlopaul/Desktop/223p/experiments/simbuild-semantic-25/templates')
# Generate shapes
handler.generate_shapes()
# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")


BuildingMOTIF does not exist, instantiating: 


<Graph identifier=N02dbc9ad90b34a85b76bdf653c439173 (<class 'rdflib.graph.Graph'>)>

In [4]:
g = Graph()
g.parse('post-processed-model.ttl', format = 'ttl')
og = clone.clone_graph(g)
inferred_graph = handler.infer(g)

Here we can see that many classes have been inferred based on our templates. 
All the zone temperatures will be labeled with the class HPFS:temp. Same for our zone temperatures.

We also infer some basic information, such as has-point. The semantic_model_builder currently uses these templates as the interface between the HPFS ontology and another ontology (i.e. brick or S223). This was done as an experimental approach for ontology alignment, but means that we must create some boilerplate templates to load data into objects.

In [29]:
for triple in (inferred_graph - og):
    if ('Ref' in str(triple[0])) or ('has-point' in str(triple[1])):
        continue 
    else:
        print(triple)

(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonCor_TZon'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#temp'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupNor_TZonHeaSet'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#heaset'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonCooSet'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#cooset'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonHeaSet'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#heaset'))
(rdflib.term.URIRef('https://BESTESTAir.urn#hvac_reaZonSou_TZon'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('urn:hpflex/shapes#temp'))
(rdflib.term.URIRef('https://BESTESTAir.urn#flo

In [30]:
list(inferred_graph[:A:HPFS['heaset']])

[rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupNor_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupEas_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupWes_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupSou_TZonHeaSet'),
 rdflib.term.URIRef('https://BESTESTAir.urn#hvac_oveZonSupCor_TZonHeaSet')]

In [32]:
# Currently we load graphs from files for LoadModel, so must serialize this
inferred_graph.serialize(f"inferred_graph.ttl")

<Graph identifier=N9d026f64e0334d5f94c00c67b2c9fe39 (<class 'rdflib.graph.Graph'>)>

LoadModel takes a filepath to a graph, a dictionary of mapping the objects you want to create, to the templates you want to create them with, the ontology used, and optionally a directory of templates.

site_info will contain dictionary of our desired templates. Each template may contain some other entities or properties. Entities and properties will be named arbitrarily based on the name of their templates. Properties are defined so that the have a value (a number, string, or external reference) and a unit. 

We want to get python objects representing each of our zones, so we will create objects based on our zone templates. These zone templates link to 3 properties for the temperature and setpoints. Each of these properties will have a value that points to the BOPTest external reference, and they will have a unit. NOTE: the units are currently incorrectly using the qudt namespace, rather than the unit namespace

In [33]:
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'zones': 'zone'}, 
                   ontology = ontology,
                   template_dir = 'templates')
site_info = loader.get_all_building_objects()

In [34]:
site_info

{'zones': [zone(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Nor', temp=Value(value=hvac_reaZonNor_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupNor_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupNor_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
  zone(name='https://BESTESTAir.urn#flo_Eas', temp=Value(value=h

In [35]:
site_info['zones']

[zone(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Nor', temp=Value(value=hvac_reaZonNor_TZon_y , unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=hvac_oveZonSupNor_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=hvac_oveZonSupNor_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K'), ),
 zone(name='https://BESTESTAir.urn#flo_Eas', temp=Value(value=hvac_reaZonEas

In [36]:
for zone in site_info['zones']:
    print("ZONE NAME IS: ", zone.name)
    print("TEMP IS: ", zone.temp)
    print("TEMP DATA IS AT EXTERNAL REFERENCE: ", zone.temp.value)
    print("HEATING SETPOINT IS: ", zone.heaset)
    print("COOLING SEPTIONT IS: ", zone.cooset)
    print("Setpoint DATA is at External References: ", zone.heaset.value,', ', zone.cooset.value)
    print('-'* 20)

ZONE NAME IS:  https://BESTESTAir.urn#flo_Wes
TEMP IS:  Value(value=hvac_reaZonWes_TZon_y , unit='http://qudt.org/schema/qudt/K')
TEMP DATA IS AT EXTERNAL REFERENCE:  hvac_reaZonWes_TZon_y 
HEATING SETPOINT IS:  Value(value=hvac_oveZonSupWes_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
COOLING SEPTIONT IS:  Value(value=hvac_oveZonSupWes_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Setpoint DATA is at External References:  hvac_oveZonSupWes_TZonHeaSet_u ,  hvac_oveZonSupWes_TZonCooSet_u
--------------------
ZONE NAME IS:  https://BESTESTAir.urn#flo_Sou
TEMP IS:  Value(value=hvac_reaZonSou_TZon_y , unit='http://qudt.org/schema/qudt/K')
TEMP DATA IS AT EXTERNAL REFERENCE:  hvac_reaZonSou_TZon_y 
HEATING SETPOINT IS:  Value(value=hvac_oveZonSupSou_TZonHeaSet_u, unit='http://qudt.org/schema/qudt/K')
COOLING SEPTIONT IS:  Value(value=hvac_oveZonSupSou_TZonCooSet_u, unit='http://qudt.org/schema/qudt/K')
Setpoint DATA is at External References:  hvac_oveZonSupSou_TZonHeaSet_u ,  